In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
# Load the dataset
with open("C:\\Users\\lenovo\\OneDrive\\Desktop\\lets grow more\\1661-0.txt", "r", encoding="utf-8") as file:
    corpus = file.read().splitlines()

In [5]:
# Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [6]:
# Create input sequences and labels
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [7]:
# Pad sequences
max_sequence_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

In [8]:
# Create predictors and labels
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [11]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [17]:
# Build the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length - 1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Use sparse_categorical_crossentropy with from_logits=True
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Example of converting one-hot encoded labels to integer labels
y_integer_labels = np.argmax(y, axis=1)

model.fit(X, y_integer_labels, epochs=10, verbose=1)

Epoch 1/10
3176/3176 [==============================] - 107s 33ms/step - loss: 6.3206 - accuracy: 0.0702
Epoch 2/10
3176/3176 [==============================] - 102s 32ms/step - loss: 5.5675 - accuracy: 0.1226
Epoch 3/10
3176/3176 [==============================] - 119s 37ms/step - loss: 5.1680 - accuracy: 0.1500
Epoch 4/10
3176/3176 [==============================] - 115s 36ms/step - loss: 4.8606 - accuracy: 0.1683
Epoch 5/10
3176/3176 [==============================] - 107s 34ms/step - loss: 4.5957 - accuracy: 0.1834
Epoch 6/10
3176/3176 [==============================] - 107s 34ms/step - loss: 4.3539 - accuracy: 0.1996
Epoch 7/10
3176/3176 [==============================] - 115s 36ms/step - loss: 4.1269 - accuracy: 0.2176
Epoch 8/10
3176/3176 [==============================] - 104s 33ms/step - loss: 3.9110 - accuracy: 0.2383
Epoch 9/10
3176/3176 [==============================] - 99s 31ms/step - loss: 3.7027 - accuracy: 0.2629
Epoch 10/10
3176/3176 [==============================] -

In [18]:
# Function to generate the next word
def generate_next_word(seed_text, num_words=1):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [20]:
# Test the model
seed_text = "I had seen little of Holmes"
predicted_text = generate_next_word(seed_text, num_words=5)
print(predicted_text)

1/1 [==============================] - 0s 28ms/step
I had seen little of Holmes and the police were in
